In [181]:
import time

import pandas as pd
from tvDatafeed import TvDatafeed, Interval

pd.set_option('display.max_rows', 10)

date_col = 'Valuation Date'
pf_name_col = 'Portfolio Name'
desc_col = 'Issuer Grouping'
sedol_col = 'SEDOL Code'
quantity_col = 'Quantity'
currency_col = 'Currency'
weight_col = 'Port. Ending Weight'
country_col = 'Country'
sector_col = 'Sector'
class_col = 'Asset Class'
exchange_col = 'Exchange'
symbol_col = 'Symbol'
security_col = 'Security No.'
index_col = 'Unnamed: 0'
today_sp_lc_col = 'SP Today lc'
today_sp_gbp_col = 'SP Today GBP'
fx_gbp_today_col = 'fx gbp today'
nav_gbp_today_col = 'nav gbp today'
fx_exchange = 'FX_IDC'

bemo_raw = ''

with open("bemo_raw.txt", 'r') as file:
    bemo_raw = file.read()

rows = bemo_raw.split('\n')


In [109]:
bemo_csv = []

headers = [date_col, pf_name_col, desc_col, sedol_col, quantity_col, currency_col, weight_col, country_col, sector_col, class_col]
countries = ["United Arab Emirates", "Turkey", "Saudi Arabia", "Czech Republic", "European Union", "Russian Federation", "United Kingdom", "Hungary", "Qatar", "Netherlands", "Kazakhstan", "Greece", "Kuwait", "PO", "TU", "United States", "Poland", "South Africa", "Romania"]
sectors = ["Financials", "Energy", "Cash", "Health Care", "Real Estate", "Consumer Discretionary", "Materials", "Consumer Staples", "Communication Services", "Industrials"]

def findX(parts, start_idx, end_idx, valid_matches):

    str = ""
    str_idx = -1

    for i in range(start_idx, end_idx, -1):
        str = (parts[i] + ' ' + str).strip()
        # print(i, parts[i])
        # print(str, str in valid_matches)
        if str in valid_matches:
            # print(f"match found at {i}: {str}")
            str_idx = i
            return str_idx
    
    return -1


def processRow(row):
    parts = row.split()

    sector_idx = findX(parts, -2, -len(parts)+5, sectors)

    if sector_idx == -1:
        print("ERROR - Sector not found")
        return

    country_idx = findX(parts, sector_idx-1, -len(parts)+5, countries)
    
    if country_idx == -1:
        print("ERROR - Country not found")
        return

    # Working backwards, as we need to find sector then country
    asset_class = parts[-1]
    sector = " ".join(parts[sector_idx:-1])
    country = " ".join(parts[country_idx:sector_idx])
    port_ending_weight = parts[country_idx-1]
    currency = parts[country_idx-2]
    quantity = parts[country_idx-3]
    sedol_code = parts[country_idx-4]
    issuer_grouping = " ".join(parts[6:country_idx-4])
    valuation_date = parts[0]
    portfolio_name = " ".join(parts[1:6])

    if country == "PO":
        country = "Poland"
    if country == "TU":
        country = "Turkey"

    row_entries = [valuation_date, portfolio_name, issuer_grouping, sedol_code, quantity, currency, port_ending_weight, country, sector, asset_class]

    return row_entries

# processRow(rows[4])

for row in rows:
    processedRow = processRow(row)
    if not processedRow:
        break
    bemo_csv.append(','.join(processedRow))


with open("bemo_holdings.csv", 'w') as file:
    file.write(','.join(headers) + '\n')
    for line in bemo_csv:
        file.write(line + '\n')

In [170]:
df = pd.read_csv("bemo_holdings.csv").drop(columns=[pf_name_col, weight_col, sector_col, class_col])
df

KeyError: "['Portfolio Name', 'Port. Ending Weight', 'Sector', 'Asset Class'] not found in axis"

In [157]:
jema_data = pd.read_csv('../jema/jema_symbols_with_vals_jun24.csv')
jema_data

,Unnamed: 0,Security Description,Security No.,Symbol,Exchange,Currency,Conv Rate,Holding Jun24,Market Value,SP JPM,SP Jun24,SP Today,fx gbp jun 24,fx gbp today,SP Jun 24 GBP,Error
0,0,THE SAUDI NATIONAL BANK,BSHYYN1,1180,TADAWUL,SAR,1,81271.00,628932.02,7.738702,36.250000,36.849998,0.208700,0.201500,7.565375,2.239738e-02
1,1,AL RAJHI BANK COMMON STOCK SAR 10,B12LZH9,1120,TADAWUL,SAR,1,32585.00,559299.16,17.164314,80.000000,89.099998,0.208700,0.201500,16.696000,2.728415e-02
2,2,FIRSTRAND LTD COMMON STOCK ZAR 1,6606996,FSR,JSE,ZAR,1,164763.00,548770.63,3.330667,7693.000000,8446.000000,0.043020,0.042750,3.309529,6.346494e-03
3,3,JPM GBP LIQUIDITY LVNAV X (DIST.),5819115,GBPGBP,FX_IDC,GBP,1,514330.73,514330.73,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000e+00
4,4,STANDARD BANK GROUP LTD COMMON STOCK ZAR 10,B030GJ7,SBK,JSE,ZAR,1,47354.00,432459.13,9.132473,21083.000000,23642.000000,0.043020,0.042750,9.069907,6.850989e-03
5,5,SAUDI ARABIAN OIL CO COMMON STOCK SAR,BJTM270,2222,TADAWUL,SAR,1,73566.00,429693.66,5.840927,27.900000,27.750000,0.208700,0.201500,5.822730,3.115487e-03
6,6,QATAR NATIONAL BANK QPSC COMMON STOCK QAR 1,6148197,QNBK,QSE,QAR,1,117376.00,372586.75,3.174301,14.650000,15.590000,0.217060,0.209570,3.179929,1.773000e-03
7,7,GOLD FIELDS LTD COMMON STOCK ZAR 50,6280215,GFI,NYSE,USD,1,31177.00,369289.12,11.844922,14.900000,16.660000,0.790540,0.763870,11.779046,5.561504e-03
8,8,POWSZECHNA KASA OSZCZEDNOSCI BANK POLSKI SA CO...,B03NGS5,PKO,PSECZ,CZK,1,29624.00,366558.54,12.373702,362.899990,333.799990,0.033726,0.033927,12.239165,1.087279e-02
9,9,EMAAR PROPERTIES PJSC COMMON STOCK AED 1,B01RM25,EMAAR,DFM,AED,1,200673.00,351812.27,1.753162,8.200000,8.470000,0.214820,0.207500,1.761524,4.769691e-03


In [162]:
# Get existing data from JEMA
for index, row in df.iterrows():

    if row[sedol_col].startswith('CASH_'):
        df.loc[index, exchange_col] = fx_exchange
        df.loc[index, symbol_col] = row[currency_col]
    else:
        try:
            sedol = row[sedol_col]
            jema_row = jema_data.loc[jema_data[] == sedol]
            jema_symbol = jema_row[symbol_col].item()
            jema_exchange = jema_row[exchange_col].item()
            df.loc[index, symbol_col] = jema_symbol
            df.loc[index, exchange_col] = jema_exchange
        except:
            # print(f"failed: {sedol} to {jema_symbol}")
            pass

df.to_csv('bemo_holdings.csv')

In [172]:
bemo = pd.read_csv('bemo_holdings_with_syms_exchs.csv').drop(columns=[index_col])
bemo

,Valuation Date,Issuer Grouping,SEDOL Code,Quantity,Currency,Country,Symbol,Exchange
0,6/30/2024,ABU DHABI COMMERCIAL BANK,6545464,1030804,AED,United Arab Emirates,ADCB,ADX
1,6/30/2024,ADNOC DRILLING COMPANY,BN12D39,2225680,AED,United Arab Emirates,ADNOCDRILL,ADX
2,6/30/2024,AED CASH(COMMITTED),CASH_AED,-7,AED,United Arab Emirates,AED,FX_IDC
3,6/30/2024,AKBANK A,B03MN70,717096,TRY,Turkey,AKBNK,BIST
4,6/30/2024,AL MOUWASAT MEDICAL SERVICES,B403QG4,18608,SAR,Saudi Arabia,4002,TADAWUL
...,...,...,...,...,...,...,...,...
73,6/30/2024,UNITED COMPANY RUSAL,BNGCVY1,572570,RUB,Russian Federation,RUAL,ALOR
74,6/30/2024,USD CASH(Alpha Committed),CASH_USD,5226829,USD,United States,USD,FX_IDC
75,6/30/2024,YANDEX NV CLASS A,B5BSZB3,82492,USD,Russian Federation,YDEX,ALOR
76,6/30/2024,YAPI VE KREDI BANKASI A,B03MZJ6,1398483,TRY,Turkey,YKBNK,BIST


In [174]:
tv = TvDatafeed()

currencies = list(bemo[currency_col].unique())
currencies_today = {}

for currency in currencies:
    if currency in currencies_today:
        continue
    try:
        data = tv.get_hist(symbol=f'{currency}GBP', exchange=fx_exchange, interval=Interval.in_daily, n_bars=100)
        currencies_today[currency] = data.iloc[-1].close.item() 
    except:
        print(f"Failed: {currency}GBP")

display(currencies_today)

you are using nologin method, data you access may be limited


{'AED': 0.20564,
 'TRY': 0.02187,
 'SAR': 0.1998,
 'PLN': 0.1982,
 'EUR': 0.84698,
 'ZAR': 0.04265,
 'CZK': 0.03377,
 'RUB': 0.008251,
 'GBP': 1.0,
 'HUF': 0.002148,
 'QAR': 0.20775,
 'USD': 0.75657,
 'KWD': 2.458,
 'RON': 0.16972}

In [187]:
tv = TvDatafeed()

for index, row in bemo.iterrows():

    symbol = row.Symbol
    exchange = row.Exchange
    currency = row.Currency
    fx_rate = currencies_today[currency]

    if exchange == fx_exchange:
        bemo.loc[index, today_sp_lc_col] = 1
        bemo.loc[index, fx_gbp_today_col] = fx_rate
        bemo.loc[index, today_sp_gbp_col] = fx_rate
        continue

    try:
        hist = tv.get_hist(
            symbol=symbol,
            exchange=exchange,
            interval=Interval.in_daily,
            n_bars=1
        )

        sp_now = hist.iloc[-1].close.item()
        bemo.loc[index, today_sp_lc_col] = sp_now

        bemo.loc[index, fx_gbp_today_col] = fx_rate
        bemo.loc[index, today_sp_gbp_col] = sp_now * fx_rate
    except:
        print(f"Failed: {symbol} - {exchange}")

    time.sleep(3)

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Failed: BID - JSE


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Failed: 0QFP - LSIN


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Failed: IMP - JSE


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Failed: MTN - JSE


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Failed: 0M69 - LSIN


In [189]:
# bemo.to_csv('bemo_with_lc_sp.csv')
pd.set_option('display.max_rows', None)
bemo

,Valuation Date,Issuer Grouping,SEDOL Code,Quantity,Currency,Country,Symbol,Exchange,SP Today lc,fx gbp today,SP Today GBP
0,6/30/2024,ABU DHABI COMMERCIAL BANK,6545464,1030804,AED,United Arab Emirates,ADCB,ADX,8.710000,0.205640,1.791124
1,6/30/2024,ADNOC DRILLING COMPANY,BN12D39,2225680,AED,United Arab Emirates,ADNOCDRILL,ADX,4.500000,0.205640,0.925380
2,6/30/2024,AED CASH(COMMITTED),CASH_AED,-7,AED,United Arab Emirates,AED,FX_IDC,1.000000,NaN,NaN
3,6/30/2024,AKBANK A,B03MN70,717096,TRY,Turkey,AKBNK,BIST,58.099998,0.021870,1.270647
4,6/30/2024,AL MOUWASAT MEDICAL SERVICES,B403QG4,18608,SAR,Saudi Arabia,4002,TADAWUL,107.000000,0.199800,21.378600
5,6/30/2024,AL RAJHI BANK,B12LZH9,307693,SAR,Saudi Arabia,1120,TADAWUL,88.700000,0.199800,17.722260
6,6/30/2024,ALDAR PROPERTIES,B0LX3Y2,1096252,AED,United Arab Emirates,ALDAR,ADX,7.070000,0.205640,1.453875
7,6/30/2024,ALLEGRO SA 144A,BMBQDF6,187194,PLN,Poland,0A5O,LSIN,36.456100,0.198200,7.225599
8,6/30/2024,ALPHA SERVICES AND HOLDINGS SA,BZ1MXR7,1423484,EUR,Greece,0RCS,LSIN,1.584000,0.846980,1.341616
9,6/30/2024,ANGLO AMERICAN PLATINUM LTD,6761000,29453,ZAR,South Africa,AMS,JSE,69557.000000,0.042650,2966.606050
